In [12]:
import sys, csv, os
import numpy as np
import pandas as pd
import pdb

In [13]:
paths = {'trial1':'',
         'trial2':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\draft2\events.csv",
         'trial8444':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604889008444_64pc\instance_1\output\events.csv",
         'trial4':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604882689569_35pc\instance_1\output\events.csv",
         'trial85a':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates85pc\simphony_model_1614483337038\instance_1\output\events.csv",
         'trial85b':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates85pc\simphony_model_1614485140334\instance_1\output\events.csv",
         'trial85c':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates85pc\simphony_model_1614526877900\instance_1\output\events.csv"
        }
#toy trial
#events_fpath = r"C:\Users\agutf\AppData\Local\Temp\simphony_model_1593922625316\instance_1\output\events.csv"
chosen_trial = 'trial8444'
#D2E60
events_fpath = paths[chosen_trial]

In [14]:
print("Reading events from: " + events_fpath)

Reading events from: C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604889008444_64pc\instance_1\output\events.csv


In [15]:
time_between_bleeds = 28
reporting_frequency = time_between_bleeds/365
#rebase_time_to_final_dose

In [16]:
print(f"Time between simulated bleeds (days): {time_between_bleeds}")

Time between simulated bleeds (days): 28


In [17]:
vaccine_trial_end_date = 1400/365.0 #external default

In [18]:
print(f"vaccine_trial_end_date (years): {vaccine_trial_end_date}")

vaccine_trial_end_date (years): 3.835616438356164


In [19]:
output_fpath = f'{events_fpath}_history_{chosen_trial}_gap{time_between_bleeds}.csv'
print(f"Output file: {output_fpath}")

Output file: C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604889008444_64pc\instance_1\output\events.csv_history_trial8444_gap28.csv


In [20]:
#events_file = open(events_file,'r')
#events = csv.DictReader(events_file)

In [21]:
#events_file.close()

In [22]:
trial_header = pd.read_csv(events_fpath, nrows=2, header=1)

In [23]:
#trial_header

In [24]:
burn_in_days = trial_header.burn_in_days[0]
vaccine_schedule = trial_header.vaccine_schedule[0]
vaccine_enrollment_launch_day = trial_header.vaccine_enrollment_launch_day


In [25]:
#vaccine_trial_end_date = 2*vaccine_trial_end_date

In [26]:
print(f'vaccine_schedule: {vaccine_schedule}')

vaccine_schedule: D2E60


In [27]:
events = pd.read_csv(events_fpath, header=4)
events.rename(columns={'Num Buddies':'total_network_size'}, inplace=True)

In [28]:
#events.tail()

In [29]:
#events[events.Event == 'enteredfollowup'].head()

In [125]:
#pd.unique(events.HCV)

In [126]:
trial_starts = events[events.Event == 'trialstarted'].copy()
trial_starts['RNA_at_start_of_trial'] = trial_starts.Info.str.contains('RNA=true')
trial_starts = trial_starts.drop(columns=['EC','L1','L2','N1','N2','C','Info','Event','DBLabel','HCV'])
trial_starts = trial_starts.rename(columns={'Time':'time_entered_trial'})

agent_baselines = {}
for row in trial_starts.iterrows():
    agent_data = row[1]
    agent = agent_data.Agent
    agent_baselines[agent] = agent_data

In [127]:
#trial_starts[trial_starts.Agent==254477147]

In [128]:
events2 = events[events.Agent.isin(agent_baselines)]

In [132]:
agent_abpos_at_enrollment = {}
agent_vaccination_time = events2[(events2.Event == 'vaccinated')].groupby(['Agent'])['Time'].first().to_dict()
agent_vaccination_time = {a: float(v) for a, v in agent_vaccination_time.items()}
for event in events2.itertuples():
    if float(event.Time) < agent_vaccination_time[event.Agent]:
        agent_abpos_at_enrollment[event.Agent] = 'abneg' if event.HCV == 'susceptible' else 'abpos'

In [133]:
for agent in agent_baselines:
    agent_baselines[agent]['ABpos_at_start_of_trial'] = agent_abpos_at_enrollment[agent]
    agent_baselines[agent]['doses_received'] = agent_vaccination_doses[agent]

In [65]:
vaccinations = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'vaccinated')]
agent_vaccination_doses = vaccinations.groupby(['Agent']).size().to_dict()

In [36]:
ever_infected = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'infected')]
infected_roster = ever_infected.groupby(['Agent']).size().to_dict()
#for agents with infections, count of the number of infections

In [37]:
#ever_recovered = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'recovered')]
#recovered_count = ever_recovered.groupby(['Agent']).size().to_dict()

In [38]:
#events2[events2.Agent==254477147]

In [39]:
#events2[events2.Agent == list(agent_baselines.keys())[2]]

In [40]:
#pd.unique(events2.Event)

In [47]:
events2[events2.Agent==1359870546].iloc[0]

Time                                         1
EC                                         NaN
Agent                               1359870546
Event                                activated
Info                                       NaN
L1                                           -
N1                                           -
L2                                           -
N2                                           -
C                                          NaN
Age                                    31.9945
Gender                                    Male
Race                                   NHWhite
Zip                                      60644
Syringe_source                              HR
HCV                                susceptible
HCV_friend_preval                            0
Age_Started                                 15
Drug_in_degree                               1
Drug_out_degree                              1
current_total_network_size                   1
Daily_injecti

In [52]:
events2[events2.Agent==300460332]

,Time,EC,Agent,Event,Info,L1,N1,L2,N2,C,...,current_total_network_size,Daily_injection_intensity,Fraction_recept_sharing,DBLabel,trial_arm,chicago_community_name,fraction_total_network_female,fraction_total_network_nh_white,fraction_total_network_inder30,mean_age_total_network
23094,1.000,NaN,300460332,activated,NaN,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,noarm,OutsideChicago,0.5,1.0,0.5,29.104110
33281,1.184,NaN,300460332,trialstarted,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.293151
33282,1.184,NaN,300460332,vaccinated,newstage=notenrolled;RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.293151
35625,1.348,NaN,300460332,vaccinated,newstage=notenrolled;RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.457534
35626,1.348,NaN,300460332,enteredfollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.457534
37229,1.425,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.534247
39019,1.501,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.610959
41032,1.578,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.687671
43232,1.655,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,1,7.052627,0.303813,3957,placebo,OutsideChicago,0.0,1.0,0.0,32.663014
45676,1.732,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,32.673973


In [45]:
#the following events do not provide new information about the RNA state or the censorship, so we drop them
# if the agent becomes infected or recovered, we will retrieve it from "infected" and "recovered" events below
#  remove to simplify the parsing
# added infectious and chronic for safety's sake
events3 = events2[~events2.Event.isin
                  (['activated','enteredfollowup','infollowup','infollowup2','vaccinated'])]

In [50]:
events3[events3.Agent==300460332]

,Time,EC,Agent,Event,Info,L1,N1,L2,N2,C,...,current_total_network_size,Daily_injection_intensity,Fraction_recept_sharing,DBLabel,trial_arm,chicago_community_name,fraction_total_network_female,fraction_total_network_nh_white,fraction_total_network_inder30,mean_age_total_network
33281,1.184,NaN,300460332,trialstarted,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.293151
67044,2.260,NaN,300460332,infected,NaN,Time_to_exposure,2.265753424657534,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,1.0,1.0,0.0,32.531507
67121,2.263,NaN,300460332,infectious,NaN,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,1.0,1.0,0.0,32.534247
68127,2.288,NaN,300460332,recovered,NaN,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,1.0,1.0,0.0,32.557534
91568,2.956,NaN,300460332,trialcompleted,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,1.0,21.043836
105106,4.712,NaN,300460332,infected,NaN,Time_to_exposure,2.4465753424657533,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,36.201370
105118,4.718,NaN,300460332,infectious,NaN,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,36.206849
105119,4.718,NaN,300460332,recovered,NaN,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,36.206849
105992,5.011,NaN,300460332,infected,NaN,Time_to_exposure,0.3041095890410959,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,36.504110
106003,5.016,NaN,300460332,infectious,NaN,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,36.509589


In [43]:
#pd.unique(events3.Event)

In [33]:
#pd.unique(events3.HCV)

In [55]:
agent_events = {}
longest_history = 0 #needed for padding the data
for agent in agent_baselines:
    a_events = events3[events3.Agent == agent][['Event','HCV','Time']]
    a_events = a_events[a_events.Time <= vaccine_trial_end_date]
    a_events = a_events.sort_values(by="Time") #just in case
    a_events = a_events.append(pd.DataFrame([['fieldworkcompleted', '-', vaccine_trial_end_date]], columns=list(['Event','HCV', 'Time'])))
    a_events["StepsFromEnrollmentStart"] = (a_events.Time - agent_baselines[agent]["time_entered_trial"])/reporting_frequency
    agent_events[agent] = a_events
    longest_history = max(longest_history, a_events.Time.iloc[-1]-a_events.Time.iloc[0])  
                          #min(vaccine_trial_end_date-a_events.Time.iloc[0], 
                          #    a_events.Time.iloc[-1]-a_events.Time.iloc[0]))

longest_history_len = longest_history/reporting_frequency

KeyError: 1835313669

In [36]:
#a_events

In [37]:
#print(longest_history)
#longest_history_len = longest_history/reporting_frequency
#print(longest_history_len)

#for agent, a_events in agent_events.items():
#    enrollment_start = agent_events[agent].Time.iloc[0]
#    if a_events['Time'].iloc[-1] < longest_history:
#        agent_events[agent] = agent_events[agent].append(pd.DataFrame([['padding', 'padding', longest_history]], columns=list(['Event','HCV', 'Time'])))
#    agent_events[agent].sort_values(by="Time", inplace=True)

In [38]:
#agent=list(agent_events.keys())[1]

In [39]:
#agent_events[agent]

In [40]:
#pd.unique(events3.Event)

In [41]:
#pd.unique(events3.HCV)

In [49]:
'''
produce a sequence [+,-,-,...,lost,lost,...,padding,padding,..]
the first status is the RNA at the start of the trial
the history records all infection episodes through the end of the trial or loss 
TODO: ensure it outputs a complete record of 9 follow-ups if infection resolves, perhaps status-censored
'''
def estimate_history(agent_id, my_events, RNA_positive_at_start_of_trial, target_length):
    current_hcv      = 'RNA_positive' if RNA_positive_at_start_of_trial else 'RNA_negative'
    last_hcv_reading = current_hcv #last known hcv status
    in_tracking      = True
    h = [] #test the correct count [current_hcv]
    #assert my_events.shape[0]>1 #at least one event in history
    if my_events.shape[0]==0:
        print(f'WARNING: history for agent {agent_id} contains just one event')
        return h, last_hcv_reading
    for i, new_event in enumerate(my_events.Event):
        if i == 0:
            if new_event != 'trialstarted':
                return [], None
            continue
        full_steps_to_new = int(my_events.StepsFromEnrollmentStart.iloc[i] - \
                                my_events.StepsFromEnrollmentStart.iloc[i-1])
        h = h + [current_hcv] * full_steps_to_new
        
        #full_steps_to_new might be less than 1, in which case we will add nothing, but the state will change
        if new_event in frozenset(['infected', 'infectious', 'chronic']) and in_tracking:
            current_hcv = 'RNA_positive'
            last_hcv_reading = current_hcv
        elif new_event == 'recovered' and in_tracking:
            current_hcv = 'RNA_negative'
            last_hcv_reading = current_hcv
        elif new_event in frozenset(['trialabandoned', 'deactivated']):
            current_hcv = 'lost_to_followup'
            in_tracking = False
        elif new_event in frozenset(['trialcompleted', 'fieldworkcompleted']): 
            break
        elif not in_tracking:
            continue
        else:
            print(new_event)
            if my_events.shape[0]==0:
                print(f'WARNING: history for agent {agent_id} has unknown event {new_event}')
                return h, last_hcv_reading
            #assert False
        #invariant: h contains all the records up to my_events.StepsFrom[i]. current_hcv is the new state
    
    h = h + ['padding']*int(target_length-len(h))
    
    #pdb.set_trace()
    #print(len(h))
    return h, last_hcv_reading

In [59]:
agent_events[agent]

,Event,HCV,Time,StepsFromEnrollmentStart
33281,trialstarted,vaccinated,1.184000,0.000000
67044,infected,exposed,2.260000,14.026429
67121,infectious,infectiousacute,2.263000,14.065536
68127,recovered,recovered,2.288000,14.391429
91568,trialcompleted,recovered,2.956000,23.099286
0,fieldworkcompleted,-,3.835616,34.565714


In [58]:
agent=300460332
estimate_history(agent,
                 agent_events[agent], 
                 agent_baselines[agent]['RNA_at_start_of_trial'], 
                 longest_history_len)

(['RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'RNA_negative',
  'padding',
  'padding',
  'padding',
  'padding',
  'padding',
  'padding',
  'padding',
  'padding',
  'padding',
  'padding',
  'padding',
  'padding',
  'padding'],
 'RNA_negative')

In [43]:
#estimate_history(list(agent_events.values())[2], 
#                 agent_baselines[agent]['RNA_at_start_of_trial'], 
#                 longest_history_len)

In [44]:
#list(agent_events.values())[2]

In [45]:
agent_histories = {}
for agent, a_events in agent_events.items():
    regular_history, last_hcv_reading = estimate_history(agent, 
                                                         a_events, 
                                                         agent_baselines[agent]['RNA_at_start_of_trial'],
                                                         longest_history_len)
    if regular_history == []: #very rare - e.g. rarely first event is "deactivated" (bug in APK)
        continue
    
    agent_baselines[agent]['notinfected_and_abpos_at_final_reading']   = \
               (agent_baselines[agent]['trial_arm'] == 'study' and last_hcv_reading == 'RNA_negative') or \
               (agent_baselines[agent]['trial_arm'] == 'placebo' and (agent in infected_roster) and last_hcv_reading == 'RNA_negative')
    #print(agent_baselines[agent]['notinfected_and_abpos_at_final_reading'])
    #print(agent_baselines[agent]['trial_arm'])
    #print(agent in infected_count)
    #print(last_hcv_reading)
    
    agent_histories[agent] = agent_baselines[agent].append(pd.Series(regular_history))

In [44]:
print('total agents parsed: %d'%len(agent_histories))
#events3[events3.Agent==1623552776]

NameError: name 'agent_histories' is not defined

In [47]:
#agent_histories[1623552776]

In [48]:
#agent_histories[agent]

In [49]:
#a_events

In [50]:
#a_events

In [51]:
#agent_histories[agent]

In [52]:
#vaccine_trial_end_date

In [53]:
output_df = pd.DataFrame.from_dict(agent_histories, orient='columns')

In [54]:
#agent_0 = agent_histories.keys()[0]
#output_df.set_index(agent_baselines[agent_0].keys() + range(longest_history), inplace=True)

In [55]:
output_df = output_df.transpose()

In [56]:
output_df.to_csv(output_fpath)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\agutf\\Sync\\repos\\apk-development\\analysis\\vaccine_cox\\candidates85pc\\simphony_model_1614526877900\\instance_1\\output\\events.csv_history_trial85c_gap28.csv'

In [ ]:
print(f'Written: {output_fpath}')

-----